In [73]:
import pandas as pd
import numpy as np

data_folder = "C:/Users/Alex H/OneDrive/Documents/BU Fall 2023/DS701/ds-livable-streets-infra/fa23-team/"

combined_demo_prop = pd.read_csv(data_folder + "combined_add_jp.csv", index_col=False)
combined_demo_prop.shape

(578, 16)

In [74]:
# Replace tract 813.01 with 804.01
combined_demo_prop = combined_demo_prop[combined_demo_prop.tract != 813.01]
print(combined_demo_prop.tract.nunique())

ct804 = pd.read_csv(data_folder + "demo_804-01.csv", index_col=False)
combined_demo_prop = pd.concat([combined_demo_prop, ct804], ignore_index=True)
print(combined_demo_prop.tract.nunique())

52
53


In [75]:
combined_income_owner = pd.read_csv(data_folder+"income_houseownership.csv", index_col=False)
combined_income_owner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654 entries, 0 to 653
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Unnamed: 0                                   654 non-null    int64  
 1   year                                         654 non-null    int64  
 2   Median income (dollars)                      574 non-null    float64
 3   Mean income (dollars)                        577 non-null    float64
 4   median_income_difference                     524 non-null    float64
 5   mean_income_difference                       528 non-null    float64
 6   census_tract                                 654 non-null    float64
 7   Total population in occupied housing units:  584 non-null    float64
 8   Owner_occupied                               584 non-null    float64
 9   Owner_occupied_Moved in 1999 or earlier      584 non-null    float64
 10  Re

In [79]:
if combined_demo_prop.tract.nunique() == combined_income_owner.census_tract.nunique():
    print("True")
else:
    print("False")

True


In [84]:
combined_income_owner = combined_income_owner.drop("Unnamed: 0", axis=1)

combined_income_owner.columns = ["year", "median_income", "mean_income", "median_income_difference", "mean_income_difference", "tract", "housed_population", "owner_occupied", "owner_occupied_pre-2000", "renter_occupied", "renter_occupied_pre-2000", "percent_owners", "percent_renters", "percent_owners_pre-2000", "percent_renters_pre-2000", "renters_owners_ratio", "renters_owners_pre-2000_ratio"]

In [85]:
convert = {'tract': str}

combined_demo_prop = combined_demo_prop.astype(convert)
combined_income_owner = combined_income_owner.astype(convert)

In [114]:
# Combine Census Data
census = pd.merge(combined_demo_prop, combined_income_owner, on=["tract", "year"], how="outer")
census = census.sort_values(['tract', 'year'])
census.shape

(654, 31)

In [119]:
census.tract = census.tract.str.replace(r'\.0$', '', regex=True)

In [120]:
dorchester = ["901", "902", "903", "904", "906", "907", "909.01",
              "910.01", "911", "912", "913", "914", "915", "916",
              "917", "918", "919", "920", "921.01", "922", "923",
              "924", "1001", "1002", "1003", "1004", "1005", "1006.01",
              "1006.03", "1007", "1008"]
roxbury = ["708", "708.01", "709", "709.01", "801", "803", "804.01",
           "805", "806.01", "813", "813.02", "814", "815", "817", "818",
           "819", "820", "821"]
jp = ["1201.04", "1202.01", "1101.05", "1101.03"]

In [126]:
# Merge with bike data
bike_data = pd.read_csv(data_folder + "analysis/tracts_bike_data.csv", index_col=False)
bike_data = bike_data.drop("Unnamed: 0", axis=1)
bike_data = bike_data.astype(convert)

census_bike = pd.merge(census, bike_data, on=['tract', 'year'], how="outer")

census_bike.shape

(775, 40)

In [127]:
bike_data.tract.nunique()

55

In [128]:
census_bike

,tract,year,total_population,male,female,median_age,race_white,race_black,race_ai_alaskan,race_asian,...,renters_owners_pre-2000_ratio,BFBL,BL,BLSL,SBL,SLM,SUP,total_built_all_lanes,cumulative_length,SLMTC
0,1001,2010,5617.0,0.481,0.519,23.9,0.063,0.638,0.012,0.018,...,0.700662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,2011,5657.0,0.480,0.520,24.5,0.109,0.619,0.014,0.003,...,0.774003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,2012,6189.0,0.451,0.549,25.2,0.074,0.701,0.004,0.015,...,1.260708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001,2013,6295.0,0.433,0.567,27.3,0.112,0.727,0.001,0.011,...,1.236902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001,2014,6371.0,0.455,0.545,31.2,0.131,0.750,0.001,0.018,...,0.946850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,924.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0.0,0.0,170.836985,0.000000,0.0,170.836985,2883.347291,0.0
771,9803.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0.0,0.0,0.000000,128.249559,0.0,128.249559,850.058818,0.0
772,9803.0,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0.0,0.0,0.000000,799.222219,0.0,799.222219,1649.281036,0.0
773,9803.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0.0,0.0,1327.766788,0.000000,0.0,1327.766788,2977.047824,0.0


In [130]:
census_bike.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775 entries, 0 to 774
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   tract                          775 non-null    object 
 1   year                           775 non-null    int64  
 2   total_population               614 non-null    float64
 3   male                           614 non-null    float64
 4   female                         614 non-null    float64
 5   median_age                     614 non-null    float64
 6   race_white                     614 non-null    float64
 7   race_black                     614 non-null    float64
 8   race_ai_alaskan                614 non-null    float64
 9   race_asian                     614 non-null    float64
 10  race_hawaiian                  614 non-null    float64
 11  race_other                     614 non-null    float64
 12  race_hispanic                  614 non-null    flo

In [129]:
census_bike.to_csv(data_folder + "census_bike_merged.csv", index=False)